In [1]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np

Update list of polymers based on raw data collected

In [56]:
polymers = pd.read_excel('Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
df_master = pd.read_pickle("database_2021_11_30.pkl")
columns = ['rid', 'smiles', 'fingerprints']
df_master = df_master[columns]

In [57]:
SMILES_df = pd.DataFrame(columns = ['Polymer', 'SMILES', 'PID'])
SMILES_df = pd.concat([SMILES_df, polymers["Homopolymer"][['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Copolymer"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 1': 'Polymer', 'SMILES 1': 'SMILES', 'PID 1':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = pd.concat([SMILES_df, polymers["Polymer Blend"].rename(columns = {'Polymer 2': 'Polymer', 'SMILES 2': 'SMILES', 'PID 2':'PID'}).loc[:,['Polymer','SMILES', 'PID']]], ignore_index = True)
SMILES_df = SMILES_df.drop_duplicates('Polymer').reset_index().drop(columns=['index'])
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [59]:
SMILES_df.to_excel('Polymer-SMILES.xlsx', sheet_name ='Polymers', index = False)

Read in updated list with SMILES to get polymer fingerprints

In [21]:
SMILES_df = pd.read_excel('Polymer-SMILES.xlsx')
IDcon = pd.read_excel('Data/polymer_master_data_v20200625.xlsx')
SMILES_df['SMILES'] = SMILES_df['SMILES'].astype(str)

In [41]:
df_master

,rid,smiles,fingerprints
0,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
1,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
2,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
3,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
4,R1000001,[*]C[*],"{'afp_C4_C4_C4': 0.3333333333333333, 'afp_C4_C..."
...,...,...,...
29807,R1014114,S=C([*])CCCC(C(C)(C)C)S[*],NaN
29808,R1014115,S=C([*])CCCC(C(C)C(C)C)S[*],NaN
29809,R1014116,S=C([*])CCCC(CC(C)(C)C)S[*],NaN
29810,R1014117,S=C([*])CCCC(C(C)C(C)(C)C)S[*],NaN


In [52]:
fp_df = pd.DataFrame()
for i in SMILES_df.index:
    if 'c' in SMILES_df.loc[i, 'SMILES'] or 'C' in SMILES_df.loc[i, 'SMILES']:
        fp = pd.DataFrame(df_master.loc[df_master['smiles'] == SMILES_df.loc[i, 'SMILES']].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = SMILES_df.loc[i, 'SMILES']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)
    elif pd.isna(SMILES_df.loc[i, 'PID']) == False:
        RID = IDcon.loc[IDcon['PID'] == SMILES_df.loc[i, 'PID'], 'RID'].values[0]
        fp =  pd.DataFrame(df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'fingerprints'], index = [i])
        fp.loc[i, 'SMILES'] = df_master.loc[df_master['rid'] == RID].reset_index(drop = True).loc[0, 'smiles']
        fp.loc[i, 'Polymer'] = SMILES_df.loc[i, 'Polymer']
        fp_df = fp_df.append(fp, ignore_index = True)

In [53]:
fp_df

,afp_C4_C4_C4,afp_C4_C4_H1,afp_H1_C4_H1,bfp_267,bfp_282,bfp_289,bfp_304,bfp_315,bfp_334,bfp_349,...,afp_C2_C4_H1,bfp_344,mfp_MQNs15,afp_C3_C3_N3,afp_C3_N3_C3,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210
0,0.444444,1.111111,0.444444,0.444444,0.444444,0.333333,1.333333,0.222222,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.583333,0.833333,0.583333,0.666667,NaN,0.250000,1.750000,NaN,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.333333,1.333333,0.333333,NaN,NaN,1.000000,1.000000,NaN,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.125000,0.375000,0.062500,NaN,0.250000,0.187500,0.750000,0.125000,0.375000,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.400000,1.200000,0.400000,0.266667,0.266667,0.600000,1.200000,0.133333,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.333333,1.333333,0.333333,NaN,NaN,1.000000,1.000000,NaN,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.266667,0.466667,0.466667,0.533333,NaN,0.200000,1.400000,NaN,0.533333,0.533333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.333333,0.666667,0.166667,NaN,NaN,0.500000,1.000000,NaN,0.666667,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.444444,1.111111,0.444444,0.444444,0.444444,0.333333,1.333333,0.222222,0.666667,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.133333,0.866667,0.433333,0.266667,NaN,0.700000,0.400000,NaN,0.400000,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
cols = list(fp_df)
cols.insert(0, cols.pop(cols.index('SMILES')))
cols.insert(0, cols.pop(cols.index('Polymer')))
fp_df = fp_df[cols].drop_duplicates('SMILES').fillna(0)

In [55]:
fp_df

,Polymer,SMILES,afp_C4_C4_C4,afp_C4_C4_H1,afp_H1_C4_H1,bfp_267,bfp_282,bfp_289,bfp_304,bfp_315,...,afp_C2_C4_H1,bfp_344,mfp_MQNs15,afp_C3_C3_N3,afp_C3_N3_C3,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210
0,poly(prop-1-ene),[*]CC([*])C,0.444444,1.111111,0.444444,0.444444,0.444444,0.333333,1.333333,0.222222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
1,polyisobutylene,[*]CC([*])(C)C,0.583333,0.833333,0.583333,0.666667,0.000000,0.250000,1.750000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2,Linear HDPE,[*]C[*],0.333333,1.333333,0.333333,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
3,polystyrene,[*]CC([*])c1ccccc1,0.125000,0.375000,0.062500,0.000000,0.250000,0.187500,0.750000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
4,poly(pent-1-ene),[*]CC([*])CCC,0.400000,1.200000,0.400000,0.266667,0.266667,0.600000,1.200000,0.133333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
6,poly(methyl methacrylate),[*]CC([*])(C)C(=O)OC,0.266667,0.466667,0.466667,0.533333,0.000000,0.200000,1.400000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
7,poly(vinylidene fluoride),[*]CC([*])(F)F,0.333333,0.666667,0.166667,0.000000,0.000000,0.500000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
9,Poly(ethyl-n-butylsilylene),[*]C[Si]([*])(CCCC)CCCC,0.133333,0.866667,0.433333,0.266667,0.000000,0.700000,0.400000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
10,Polycarbosilane,[*]C[Si]([*])(C)C,0.000000,0.000000,0.583333,0.666667,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
11,poly(p-hydroxystyrene),[*]CC([*])c1ccc(O)cc1,0.117647,0.352941,0.058824,0.000000,0.235294,0.176471,0.705882,0.117647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000


In [214]:
fp_df.to_excel('Polymer-SMILES-FP.xlsx', sheet_name ='Polymers')